In [ ]:
# Import modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from torch.utils.data import Subset
import math

In [3]:
def set_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if torch.cuda.is_available():
        print(f'Using device: {device}')
        print(f'GPU: {torch.cuda.get_device_name(0)}')
    else:
        print(f'Using device: {device}')

    return device

In [4]:
def norm_data(name):
    df = pd.read_csv(name)
    ndf = pd.DataFrame()
    range_mm={
        'V': {'min':df['V'].min()*0.8, 'max': df['V'].max()*1.2},
        'E': {'min':df['E'].min()*0.8, 'max': df['E'].max()*1.2},
        'VF': {'min':df['VF'].min()*0.8, 'max': df['VF'].max()*1.2},
        'VA': {'min':df['VA'].min()*0.8, 'max': df['VA'].max()*1.2},
        'VB': {'min':df['VB'].min()*0.8, 'max': df['VB'].max()*1.2},
        'CFLA': {'min':0, 'max': df['CFLA'].max()*1.2},
        'CALA': {'min':0, 'max': df['CALA'].max()*1.2},
        'CBLA': {'min':0, 'max': df['CBLA'].max()*1.2},
        'CFK': {'min':0, 'max': df['CFK'].max()*1.2},
        'CAK': {'min':0, 'max': df['CAK'].max()*1.2},
        'CBK': {'min':0, 'max': df['CBK'].max()*1.2},
        'I': {'min':0, 'max': df['I'].max()*1.2},
    }

    ndf['exp'] = df['exp']; ndf['t'] = df['t']

    for col in ['V', 'E', 'VF', 'VA', 'VB', 'CFLA', 'CALA', 'CBLA', 'CFK', 'CAK', 'CBK', 'I']:
        if col in range_mm:
            ndf[col] = (df[col] - range_mm[col]['min'])/(range_mm[col]['max'] - range_mm[col]['min'])
        else:
            ndf[col] = df[col]
    return ndf

In [5]:
def seq_data_const(ndf):
    sequences = []
    feature_cols = ['V', 'E', 'VF', 'VA', 'VB', 'CFLA', 'CALA', 'CBLA', 'CFK', 'CAK', 'CBK', 'I']
    
    for exp in ndf['exp'].unique():
        exp_data = ndf[ndf['exp'] == exp].sort_values(by='t')
        sequences.append(exp_data[feature_cols].values)
    
    return sequences

In [6]:
def padded_sequences(sequences):
    max_seq_len = max([len(seq) for seq in sequences])
    seq_len = [len(seq) for seq in sequences]
    padded_sequences = pad_sequence([torch.tensor(seq) for seq in sequences], batch_first=True, padding_value=-1)

    return padded_sequences, seq_len, max_seq_len

In [7]:
def gen_dataset(pad_seq, seq_len):
    input_tensor= pad_seq.float()
    seq_len_tensor= torch.tensor(seq_len)

    device = set_device()
    input_tensor = input_tensor.to(device)
    seq_len_tensor = seq_len_tensor.to(device)

    dataset = TensorDataset(input_tensor, seq_len_tensor)
    return dataset

In [8]:
def kfold_dataloaders(dataset, k_folds=5, batch_size=8, random_state=42):
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=random_state)
    dataloaders = []
    batch_size = math.ceil(len(dataset)/k_folds)
    
    for fold, (train_indices, val_indices) in enumerate(kfold.split(range(len(dataset)))):
        print(f"Fold {fold + 1}: Train size = {len(train_indices)}, Val size = {len(val_indices)}")
        
        # Create subsets for train and validation
        train_subset = Subset(dataset, train_indices)
        val_subset = Subset(dataset, val_indices)
        
        # Create DataLoaders
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        dataloaders.append((train_loader, val_loader))
    return dataloaders

In [ ]:
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, seq_len):
        from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
        
        # 패딩된 시퀀스를 pack하여 효율적 처리
        packed_input = pack_padded_sequence(x, seq_len, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        
        # 다시 패딩된 형태로 복원
        lstm_out, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        norm = self.layer_norm(lstm_out)
        return self.dropout(norm)

In [10]:
class MLPDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=2, num_nodes=None, dropout = 0.3):
        super().__init__()

        if num_nodes is None:
            num_nodes = hidden_size
        
        self.layers = nn.ModuleList()

    # 첫 번째 레이어: hidden_size → num_nodes
        self.layers.append(nn.Linear(hidden_size, num_nodes))
        self.layers.append(nn.LayerNorm(num_nodes))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Dropout(dropout))

        # 중간 은닉층들: num_nodes → num_nodes
        for i in range(num_layers - 1):
            self.layers.append(nn.Linear(num_nodes,num_nodes))
            self.layers.append(nn.LayerNorm(num_nodes))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.Dropout(dropout))

        # 마지막 출력층: num_nodes → output_size
        self.layers.append(nn.Linear(num_nodes, output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class StateUpdateLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, mlp_output, cur_state):
        V = cur_state[..., 0]; E = cur_state[..., 1]
        VF = cur_state[..., 2]; VA = cur_state[..., 3]; VB = cur_state[..., 4]
        CFLA = cur_state[..., 5]; CALA = cur_state[..., 6]; CBLA = cur_state[..., 7]
        CFK = cur_state[..., 8]; CAK = cur_state[..., 9]; CBK = cur_state[..., 10]
        I = cur_state[..., 11]

        NFLA = CFLA * VF; NALA = CALA * VA; NBLA = CBLA * VB
        NFK = CFK * VF; NAK = CAK * VA; NBK = CBK * VB

        dVA = mlp_output[...,0]
        dVB = mlp_output[...,1]
        dNALA = mlp_output[...,2]
        dNBLA = mlp_output[...,3]
        dNAK = mlp_output[...,4]
        dNBK = mlp_output[...,5]

        nVF = VF - dVA - dVB; nVA = VA + dVA; nVB = VB + dVB
        nNFLA = NFLA - dNALA - dNBLA; nNALA = NALA + dNALA; nNBLA = NBLA + dNBLA
        nNFK = NFK - dNAK - dNBK; nNAK = NAK + dNAK; nNBK = NBK + dNBK
        
        # 부피 clamp 후 농도 계산 (division by zero 방지)
        nVF = torch.clamp(nVF, min=1e-8)
        nVA = torch.clamp(nVA, min=1e-8)
        nVB = torch.clamp(nVB, min=1e-8)
        
        nCFLA = nNFLA / nVF; nCALA = nNALA / nVA; nCBLA = nNBLA / nVB
        nCFK = nNFK / nVF; nCAK = nNAK / nVA; nCBK = nNBK / nVB

        nCFLA = torch.clamp(nCFLA, min=0)
        nCALA = torch.clamp(nCALA, min=0)
        nCBLA = torch.clamp(nCBLA, min=0)
        nCFK = torch.clamp(nCFK, min=0)
        nCAK = torch.clamp(nCAK, min=0)
        nCBK = torch.clamp(nCBK, min=0)
        nI = mlp_output[...,6]

        new_state = torch.cat([V, E, nVF, nVA, nVB, nCFLA, nCALA, nCBLA, nCFK, nCAK, nCBK, nI], dim=-1)
        
        return new_state

In [ ]:
class BMEDSeq2SeqModel(nn.Module):
    def __init__(self, lstm_params, mlp_params):
        super().__init__()
        self.lstm_encoder = LSTMEncoder(**lstm_params)
        self.mlp_decoder = MLPDecoder(**mlp_params)
        self.mass_balance_layer = StateUpdateLayer()

    def forward(self, x, seq_len):
        # Teacher Forcing: 전체 시퀀스로 다음 상태들 예측
        
        # LSTM으로 시계열 패턴 학습
        lstm_out = self.lstm_encoder(x, seq_len)
        
        # MLP로 상태 변화량 예측
        mlp_out = self.mlp_decoder(lstm_out)
        
        # 물리적 제약 조건 적용하여 다음 상태 계산
        next_states = self.mass_balance_layer(mlp_out, x)
        
        return next_states

In [1]:


def masked_mse_loss(predictions, targets, seq_lengths):
    """
    패딩된 시퀀스에 대한 마스크 적용 MSE 손실 함수
    
    Args:
        predictions: 모델 예측값 [batch_size, seq_len, features]
        targets: 실제 타겟값 [batch_size, seq_len, features]  
        seq_lengths: 각 시퀀스의 실제 길이 [batch_size]
    
    Returns:
        masked_loss: 패딩 부분을 제외한 평균 MSE 손실
    """
    batch_size, max_len, features = predictions.shape
    
    # 마스크 생성: 실제 시퀀스 길이만큼만 True
    mask = torch.arange(max_len)[None, :] < seq_lengths[:, None]
    mask = mask.float().to(predictions.device)
    
    # 각 요소별 MSE 계산 (reduction='none')
    loss = F.mse_loss(predictions, targets, reduction='none')
    
    # 마스크 적용하여 패딩 부분 제거
    masked_loss = (loss * mask.unsqueeze(-1)).sum() / (mask.sum() * features)
    
    return masked_loss

In [2]:
def prepare_teacher_forcing_data(input_sequences, seq_lengths):
    """
    Teacher Forcing을 위한 입력-타겟 데이터 준비
    
    Args:
        input_sequences: 전체 시퀀스 [batch_size, seq_len, features]
        seq_lengths: 각 시퀀스의 실제 길이 [batch_size]
    
    Returns:
        inputs: [t0, t1, ..., t_{n-1}] 현재 상태들
        targets: [t1, t2, ..., t_n] 다음 상태들  
        target_seq_lengths: 타겟 시퀀스 길이 (1씩 감소)
    """
    # 입력: 마지막 시점 제외 [:-1]
    inputs = input_sequences[:, :-1, :]
    
    # 타겟: 첫 번째 시점 제외 [1:]  
    targets = input_sequences[:, 1:, :]
    
    # 타겟 시퀀스 길이는 1씩 감소 (마지막 시점 예측 불가)
    target_seq_lengths = torch.clamp(seq_lengths - 1, min=1)
    
    return inputs, targets, target_seq_lengths

In [13]:
df = pd.read_csv('BMED_DATA_AG.csv')
feature_cols = ['V', 'E', 'VF', 'VA', 'VB', 'CFLA', 'CALA', 'CBLA', 'CFK', 'CAK', 'CBK', 'I']
ndf = norm_data('BMED_DATA_AG.csv')
seq = seq_data_const(ndf)
pad_seq,seq_len,max_seq_len = padded_sequences(seq)
dataset = gen_dataset(pad_seq, seq_len)
dataloaders = kfold_dataloaders(dataset, k_folds=5, batch_size=8, random_state=42)
print(f"\nCreated {len(dataloaders)} fold dataloaders")
print(f"Each fold contains (train_loader, val_loader) tuple")

Using device: cuda
GPU: NVIDIA GeForce RTX 4080 SUPER
Fold 1: Train size = 31, Val size = 8
Fold 2: Train size = 31, Val size = 8
Fold 3: Train size = 31, Val size = 8
Fold 4: Train size = 31, Val size = 8
Fold 5: Train size = 32, Val size = 7

Created 5 fold dataloaders
Each fold contains (train_loader, val_loader) tuple
